### Request to API Last.fm

Let's address to the API of the leading music portal Last.fm, which is engaged in scrobbling (reading) songs from the user's player and keeps statistics of their listening.

Our targets:
- receive statistics of the most listened performers and compositions on the territory of Moldova and form a dataframe suitable for visualization
- compare popular performers with popular performers in other countries
- test the order of interaction with the API provided by the site Last.fm

In [21]:
#import libraries

import requests
import json
import pandas as pd

### Best Performers

#### Moldova best performers

The request will go to the url. The address is provided in the API documentation. First, let's write it manually. Take the country (Moldova) and api_key, which we received by registration.

In [22]:
url = 'http://ws.audioscrobbler.com/2.0/?method=geo.gettopartists&country=moldova&api_key=2689b481c219016f770446b53238ea94&format=json'

We will make a request using the requests library. We will wait for a response from the server.

In [23]:
response = requests.get(url = url)
response

<Response [200]>

In [24]:
#read the data by json module in json format

moldova_data = json.loads(response.content)

Let's form a dataframe from the resulting json response. For this put it in the Data Frame method, clear it of unnecessary columns and change the beginning of the indexes by one.

In [25]:
#create the dataframe and clean it

data_md = pd.DataFrame(moldova_data['topartists']['artist'])
data_md.drop(columns = ['image', 'streamable', 'mbid', 'url'], inplace=True)
data_md.index = [i for i in range(1, len(data_md) + 1)]
data_md['rate'] = data_md.index
data_md.head(10)

,name,listeners,rate
1,Radiohead,5395059,1
2,Red Hot Chili Peppers,5176689,2
3,Metallica,3316349,3
4,Muse,4529299,4
5,Queen,4808158,5
6,Coldplay,6259645,6
7,Nirvana,4984920,7
8,Linkin Park,4519669,8
9,Lana Del Rey,2699865,9
10,Moby,2857252,10


***Radiohead***, ***Read Hot Chili Peppers*** and ***Metallica*** are on the top Moldavian listeners, which using Last.fm as before. (listeners mentioned for all world listeners, not specified on moldovans)

#### Russian and Romanian best performers

Let's compare Moldova's audition data with Russian and Romanian auditions.

##### Romania

We will also write the parameters manually for Romania.

In [26]:
url2 = 'http://ws.audioscrobbler.com/2.0/?method=geo.gettopartists&country=romania&api_key=2689b481c219016f770446b53238ea94&format=json'

In [27]:
response2 = requests.get(url = url2)
romania_data = json.loads(response2.content)

In [28]:
#create the datafreme, clean it and create the rating columns

data_rom = pd.DataFrame(romania_data['topartists']['artist'])
data_rom.drop(columns = ['image', 'streamable', 'mbid', 'url'], inplace=True)
data_rom.index = [i for i in range(1, len(data_rom) + 1)]
data_rom['rate'] = data_rom.index
data_rom.head(10)

,name,listeners,rate
1,Radiohead,5395059,1
2,Pink Floyd,3612930,2
3,Queen,4808158,3
4,Coldplay,6259645,4
5,Red Hot Chili Peppers,5176689,5
6,Metallica,3316349,6
7,David Bowie,3901461,7
8,Arctic Monkeys,4319211,8
9,Massive Attack,2784241,9
10,Lana Del Rey,2699865,10


Let's make a comparison also for the Russian Federation. Let's take a different approach: we'll write the parameters through the dictionary, not directly in the url.

In [29]:
#create the dict "params", create the dataframe

params = {
    'api_key':'2689b481c219016f770446b53238ea94',
    'country':'Russian Federation',
}
url_3 = 'http://ws.audioscrobbler.com/2.0/?method=geo.gettopartists&format=json'
response3 = requests.get(url = url_3, params = params)
russia_data = json.loads(response3.content)
data_rus = pd.DataFrame(russia_data['topartists']['artist'])
data_rus.drop(columns = ['image', 'streamable', 'mbid', 'url'], inplace=True)
data_rus.index = [i for i in range(1, len(data_rus) + 1)]
data_rus['rate'] = data_rus.index
data_rus.head(10)

,name,listeners,rate
1,Linkin Park,4519669,1
2,Red Hot Chili Peppers,5176689,2
3,Queen,4808158,3
4,Radiohead,5395059,4
5,Depeche Mode,3033845,5
6,Nirvana,4984920,6
7,Lana Del Rey,2699865,7
8,Muse,4529299,8
9,Coldplay,6259645,9
10,The Prodigy,2407587,10


#### Summary table for the best performers of Moldova, Romania and Russia

The next step will be justified in terms of eternal disputes. Let's see how the musical tastes of millennials differ (this is just an assumption that last.fm is outdated) in Moldova, Romania and the Russian Federation.

In [30]:
#join out tables into one by "merge" method, next clean the result dataframe

rating = pd.merge(pd.merge(data_md,data_rom,on='name', how='left'),data_rus,on='name', how='left')
rating.drop(columns=['listeners_x', 'listeners_y', 'listeners'], inplace=True)
rating.rename(columns={'rate_x':'moldova', 'rate_y':'romania', 'rate':'russia'}, inplace=True)
rating

,name,moldova,romania,russia
0,Radiohead,1,1.0,4.0
1,Red Hot Chili Peppers,2,5.0,2.0
2,Metallica,3,6.0,15.0
3,Muse,4,12.0,8.0
4,Queen,5,3.0,3.0
5,Coldplay,6,4.0,9.0
6,Nirvana,7,15.0,6.0
7,Linkin Park,8,24.0,1.0
8,Lana Del Rey,9,10.0,7.0
9,Moby,10,26.0,11.0


Let's have a little fun and find out who's to be closer to Moldova by musical tastes. We will do this without going into calculations and metrics, just using the example of the first fifty musicians and deduce the sum of the missing performers from the rating of Moldova in other ratings.

In [31]:
rating.isnull().sum()

name        0
moldova     0
romania    11
russia      9
dtype: int64

11 groups are missing in Romanian and 9 in Russian. In this extemporaneous duel, the Russian Federation was closer.

### Best Tracks

Now let's move on to determining the most listened tracks on the territory of Moldova, which were scrobble using last.fm.

In [32]:
params_tracks = {
    'api_key': '2689b481c219016f770446b53238ea94',
    'country': 'Moldova',
    'limit':25
}

Next, we will make a request, output the contained content in json format and form a dataframe from it. In general, let's repeat our actions of the performers stage.

In [33]:
response_md = requests.get(url = 'http://ws.audioscrobbler.com/2.0/?method=geo.gettoptracks&format=json', params=params)
response_md

<Response [200]>

In [34]:
md_content = json.loads(response_md.content)
md_tracks = pd.DataFrame(md_content['tracks']['track'])
md_tracks.drop(columns = ['image', 'streamable', 'url','@attr', 'mbid'], inplace=True)
md_tracks.index = [i for i in range(1, len(md_tracks) + 1)]
md_tracks.head()

,name,duration,listeners,artist
1,Smells Like Teen Spirit,301,2520039,"{'name': 'Nirvana', 'mbid': '9282c8b4-ca0b-4c6..."
2,Seven Nation Army,449,1830041,"{'name': 'The White Stripes', 'mbid': '11ae9fb..."
3,Numb,191,1570710,"{'name': 'Linkin Park', 'mbid': 'f59c5520-5f46..."
4,Intro,127,1253972,"{'name': 'The xx', 'mbid': 'c5c2ea1c-4bde-4f4d..."
5,Feel Good Inc.,221,1942725,"{'name': 'Gorillaz', 'mbid': 'e21857d5-3256-45..."


In [35]:
#clean the artist information

md_tracks['artist'] = md_tracks.artist.apply(lambda x: x['name'])
md_tracks.head()

,name,duration,listeners,artist
1,Smells Like Teen Spirit,301,2520039,Nirvana
2,Seven Nation Army,449,1830041,The White Stripes
3,Numb,191,1570710,Linkin Park
4,Intro,127,1253972,The xx
5,Feel Good Inc.,221,1942725,Gorillaz


The most popular songs in Moldova by Last.fm data are ***Smels Like Teen Spirit***, ***Seven Nation Army*** and ***Numb***.